In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+ '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char

{0: ' ',
 1: '+',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

In [7]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [8]:
print(questions[:5], expected[:5])

['62+9   ', '287+2  ', '6+884  ', '12+86  ', '594+7  '] ['71  ', '289 ', '890 ', '98  ', '601 ']


In [9]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [10]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 12)
(18000, 4, 12)
Validation Data:
(2000, 7, 12)
(2000, 4, 12)
Testing Data:
(60000, 7, 12)
(60000, 4, 12)


In [11]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False False False False False
    True]
  [False False False False  True False False False False False False
   False]
  [False False False False False False False False  True False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False  True False False False False False
   False]
  [False False False False False False False  True False False False
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False False False False False False False False False
    True]
  [False False False False False False False False False False  True
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False  True False False False
   False]
  [False False False False False  True False False False False False
   False]
  [ True False False False False False Fal

In [12]:
print('Build model...')
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [13]:
for iteration in range(10):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=10,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Instructions for updating:
Use tf.cast instead.
Train on 18000 samples, validate on 2000 samples
Epoch 1/10
18000/18000 [==============================] - 4s 236us/step - loss: 2.0241 - acc: 0.2891 - val_loss: 1.8853 - val_acc: 0.3204
Epoch 2/10
18000/18000 [==============================] - 3s 175us/step - loss: 1.8556 - acc: 0.3314 - val_loss: 1.8407 - val_acc: 0.3288
Epoch 3/10
18000/18000 [==============================] - 3s 176us/step - loss: 1.8205 - acc: 0.3315 - val_loss: 1.8035 - val_acc: 0.3351
Epoch 4/10
18000/18000 [==============================] - 3s 179us/step - loss: 1.7897 - acc: 0.3397 - val_loss: 1.7700 - val_acc: 0.3482
Epoch 5/10
18000/18000 [==============================] - 4s 195us/step - loss: 1.7482 - acc: 0.3572 - val_loss: 1.7249 - val_acc: 0.3657
Epoch 6/10
18000/18000 [==============================] - 4s 250us/step - loss: 1.6741 - acc: 0.3766 - val_loss: 1.6330 - val_acc: 0.3849
Epoch 7/10


Epoch 5/10
18000/18000 [==============================] - 4s 244us/step - loss: 0.1963 - acc: 0.9576 - val_loss: 0.2403 - val_acc: 0.9281
Epoch 6/10
18000/18000 [==============================] - 4s 232us/step - loss: 0.1728 - acc: 0.9668 - val_loss: 0.2388 - val_acc: 0.9271
Epoch 7/10
18000/18000 [==============================] - 4s 227us/step - loss: 0.1603 - acc: 0.9698 - val_loss: 0.2279 - val_acc: 0.9307
Epoch 8/10
18000/18000 [==============================] - 4s 239us/step - loss: 0.1513 - acc: 0.9704 - val_loss: 0.2134 - val_acc: 0.9340
Epoch 9/10
18000/18000 [==============================] - 4s 232us/step - loss: 0.1398 - acc: 0.9740 - val_loss: 0.1924 - val_acc: 0.9440
Epoch 10/10
18000/18000 [==============================] - 4s 233us/step - loss: 0.1323 - acc: 0.9747 - val_loss: 0.1984 - val_acc: 0.9389
Q 15+41   T 56   ☑ 56  
Q 655+799 T 1454 ☑ 1454
Q 778+49  T 827  ☑ 827 
Q 940+81  T 1021 ☑ 1021
Q 790+45  T 835  ☑ 835 
Q 58+571  T 629  ☑ 629 
Q 542+21  T 563  ☑ 563 
Q 7

18000/18000 [==============================] - 4s 233us/step - loss: 0.0110 - acc: 0.9995 - val_loss: 0.0774 - val_acc: 0.9735
Epoch 2/10
18000/18000 [==============================] - 4s 245us/step - loss: 0.1038 - acc: 0.9652 - val_loss: 0.2390 - val_acc: 0.9175
Epoch 3/10
18000/18000 [==============================] - 5s 251us/step - loss: 0.0539 - acc: 0.9842 - val_loss: 0.1001 - val_acc: 0.9649
Epoch 4/10
18000/18000 [==============================] - 5s 258us/step - loss: 0.0203 - acc: 0.9972 - val_loss: 0.0695 - val_acc: 0.9771
Epoch 5/10
18000/18000 [==============================] - 4s 240us/step - loss: 0.0101 - acc: 0.9998 - val_loss: 0.0638 - val_acc: 0.9791
Epoch 6/10
18000/18000 [==============================] - 4s 239us/step - loss: 0.0086 - acc: 0.9999 - val_loss: 0.0634 - val_acc: 0.9803
Epoch 7/10
18000/18000 [==============================] - 4s 243us/step - loss: 0.0080 - acc: 0.9999 - val_loss: 0.0604 - val_acc: 0.9806
Epoch 8/10
18000/18000 [=====================

In [14]:
Right = 0
Wrong = 0
for i in range(10000):
    ind = np.random.randint(0, len(test_x))
    rowx, rowy = test_x[np.array([i])], test_y[np.array([i])]
    preds = model.predict_classes(rowx, verbose=0)
    q = ctable.decode(rowx[0])
    correct = ctable.decode(rowy[0])
    guess = ctable.decode(preds[0], calc_argmax=False)
    
    #print('Q', q[::-1] if REVERSE else q, end=' ')
    #print('T', correct, end=' ')

    if correct == guess:
        Right += 1
    else:
        Wrong += 1
print(Right/10000)

0.9272
